In [1]:
import pandas as pd
import numpy as np

In [65]:
stats = pd.read_csv(r'NBA_scraping/data_cleaning/stats_merged.csv')



In [71]:
years = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13',
         '2011-12', '2010-11', '2009-10', '2008-09']
def ewm_year(df, season):
    merged = []
    for i in season:
        stats_x = df.loc[df['SEASON_YEAR']== i]        
        stats_x['GAME_DATE'] = pd.to_datetime(stats_x['GAME_DATE'])
        stats_x.set_index(['GAME_DATE', 'HOME', 'AWAY'], inplace=True) 
        stats_x = stats_x.sort_index()
        stats_x =stats_x.groupby('TEAM_ABBREVIATION').ewm(span=10, min_periods=10).mean()
        stats_x = stats_x.shift(1)
        stats_x = stats_x.drop('Unnamed: 0', 1)
        merged.append(stats_x)
    merged = pd.concat(merged)
    merged.to_csv('data.csv')

#ewm_year(stats,years)

<ipython-input-71-f60a0ef4efaa>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_x['GAME_DATE'] = pd.to_datetime(stats_x['GAME_DATE'])


In [72]:
stats_ewm = pd.read_csv('data.csv')

In [77]:
stats_col = stats_ewm.reset_index()
stats_home = stats_col
stats_home = stats_home[stats_home.TEAM_ABBREVIATION == stats_home.HOME]
stats_away = stats_col
stats_away = stats_away[stats_away.TEAM_ABBREVIATION == stats_away.AWAY]

In [78]:
stats_home.set_index(['GAME_DATE', 'HOME', 'AWAY'], inplace=True)
stats_away.set_index(['GAME_DATE', 'HOME', 'AWAY'], inplace=True)
stats_home = stats_home.drop('TEAM_ABBREVIATION', 1)
stats_away = stats_away.drop('TEAM_ABBREVIATION', 1)

# Subtract stats_home - stats_away if they have the same index
stats_sub = stats_home - stats_away
stats_sub = stats_home - stats_away
stats_sub = stats_sub.drop('index', 1)
stats_sub.head()

AST       BLK      DREB    FG_PCT   FG3_PCT  \
GAME_DATE  HOME AWAY                                                     
2008-10-28 BOS  CLE  -0.053016  0.008363  0.014856 -0.000706  0.000401   
           CHI  MIL  -0.001185 -0.004342 -0.058237 -0.000422 -0.001997   
           LAL  POR  -0.081911  0.001941  0.027199 -0.001031 -0.001619   
2008-10-29 DET  IND   0.052314  0.020826 -0.020202  0.000044  0.000531   
           GSW  NOP  -0.026685  0.038596  0.004095  0.000155 -0.000702   

                          FG3A      FG3M       FGA       FGM    FT_PCT  ...  \
GAME_DATE  HOME AWAY                                                    ...   
2008-10-28 BOS  CLE   0.096037  0.040449 -0.122704 -0.119029 -0.000583  ...   
           CHI  MIL  -0.150581 -0.082637 -0.046897 -0.069318  0.001075  ...   
           LAL  POR   0.030843 -0.018690 -0.098342 -0.127083  0.000008  ...   
2008-10-29 DET  IND   0.045038  0.029248  0.036809  0.016741 -0.000480  ...   
           GSW  NOP  -0.142057 -0.061073 -0.022442  0.015198 -0.000725  ...   

                      OPP_OREB_PCT  OPP_TOV_PCT  OPP_PTS_2ND_CHANCE  \
GAME_DATE  HOME AWAY                                                  
2008-10-28 BOS  CLE      -0.312678    -0.363303           -0.209151   
           CHI  MIL       0.613169     0.496412            0.069505   
           LAL  POR      -0.466203    -0.738745            0.105207   
2008-10-29 DET  IND       0.459095     0.859542            0.857839   
           GSW  NOP       0.453213    -0.549360            0.356860   

                      OPP_PTS_FB  OPP_PTS_OFF_TOV  OPP_PTS_PAINT  \
GAME_DATE  HOME AWAY                                               
2008-10-28 BOS  CLE    -0.423778         0.789898      -0.144468   
           CHI  MIL    -0.288291        -0.217313      -0.705516   
           LAL  POR    -0.474610         0.209149       0.163506   
2008-10-29 DET  IND     0.114531         0.678086       1.208946   
           GSW  NOP    -0.772434        -0.993815       0.341135   

                      PTS_2ND_CHANCE    PTS_FB  PTS_OFF_TOV  PTS_PAINT  
GAME_DATE  HOME AWAY                                                    
2008-10-28 BOS  CLE        -0.586128 -0.998246    -0.510353   0.080359  
           CHI  MIL         0.099041  0.649660     0.748712   0.551180  
           LAL  POR        -1.103748 -2.045123    -0.880749  -0.891252  
2008-10-29 DET  IND        -0.141868  1.501992     0.699618   0.158965  
           GSW  NOP         0.222531  0.064374    -0.162450  -0.059981  

[5 rows x 66 columns]

In [79]:
stats_sub.to_csv(r'stats_diff_ewm.csv')